# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,muskogee,35.5834,-95.3336,24.12,79,0,4.63,US,1726542236
1,1,san antonio de pale,-1.4014,5.6325,24.53,85,100,6.71,GQ,1726542237
2,2,oranjemund,-28.5500,16.4333,11.48,93,29,3.95,NaN,1726542238
3,3,invercargill,-46.4000,168.3500,7.23,67,91,4.02,NZ,1726542240
4,4,tobol'sk,58.2000,68.2667,7.84,64,0,2.29,RU,1726542241


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    color="City",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    cmap="viridis",
    size=30,
    hover_cols=['Humidity']
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[city_data_df['Cloudiness'] == 0]

# Drop any rows with null values
cleaned_cities_df = filtered_cities_df.dropna().reset_index(drop=True)

# Display sample data
print(cleaned_cities_df)

    City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0         0     muskogee  35.5834  -95.3336     24.12        79           0   
1         4     tobol'sk  58.2000   68.2667      7.84        64           0   
2        12       touros  -5.1989  -35.4608     24.55        78           0   
3        14  saint marys  30.7305  -81.5465     23.34        86           0   
4        18       bethel  41.3712  -73.4140     16.08        87           0   
..      ...          ...      ...       ...       ...       ...         ...   
84      572        moree -29.4667  149.8500     20.50        30           0   
85      574         arar  30.9753   41.0381     24.01        43           0   
86      575          sur  22.5667   59.5289     29.88        56           0   
87      578       yichun  47.7000  128.9000     21.36        36           0   
88      583      brandon  27.9378  -82.2859     26.95        93           0   

    Wind Speed Country        Date  
0         4.63

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df)

                    City Country      Lat       Lng  Humidity Hotel Name
0               muskogee      US  35.5834  -95.3336        79           
1    san antonio de pale      GQ  -1.4014    5.6325        85           
2             oranjemund     NaN -28.5500   16.4333        93           
3           invercargill      NZ -46.4000  168.3500        67           
4               tobol'sk      RU  58.2000   68.2667        64           
..                   ...     ...      ...       ...       ...        ...
590              trencin      SK  48.8333   18.2500        98           
591             tixkokob      MX  21.0000  -89.4000        88           
592             pimentel      PE  -6.8367  -79.9342        82           
593           luchegorsk      RU  46.4764  134.2569        57           
594               namibe      AO -15.1961   12.1522        92           

[595 rows x 6 columns]


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
geoapify_key = "5fd7c4d3a83d4503ac82e2b71db891fa"

params={
    "categories":"accommodation.hotel",
    "apikey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']


    # Add the current city's latitude and longitude to the params dictionary
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "apiKey": geoapify_key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
muskogee - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
oranjemund - nearest hotel: Op My Stoep
invercargill - nearest hotel: The Grand
tobol'sk - nearest hotel: Ямская
lihue - nearest hotel: Kauai Palms
boyolangu - nearest hotel: Crown Victoria
blackmans bay - nearest hotel: Villa Howden
nagqu - nearest hotel: Naqu Hotel
labrador city - nearest hotel: Carol Inn
port elizabeth - nearest hotel: Waterford Hotel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
touros - nearest hotel: Pousada Atlântico
sitka - nearest hotel: Aspen Suites Hotel
saint marys - nearest hotel: Cumberland Kingsbay Lodge
ribeira grande - nearest hotel: Hospedaria JSF
adamstown - nearest hotel: No hotel found
karkira - nearest hotel: استراحة دريم هاوس
bethel - nearest hotel: Hampton Inn Danbury
olonkinbyen - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
itupiranga - nearest hotel: No hotel found
verdal - nearest hotel: Verdal H

,City,Country,Lat,Lng,Humidity,Hotel Name
0,muskogee,US,35.5834,-95.3336,79,No hotel found
1,san antonio de pale,GQ,-1.4014,5.6325,85,No hotel found
2,oranjemund,NaN,-28.5500,16.4333,93,Op My Stoep
3,invercargill,NZ,-46.4000,168.3500,67,The Grand
4,tobol'sk,RU,58.2000,68.2667,64,Ямская
...,...,...,...,...,...,...
590,trencin,SK,48.8333,18.2500,98,Liečebno - rehabilitačné stredisko ZVJS
591,tixkokob,MX,21.0000,-89.4000,88,Hacienda Katanchel
592,pimentel,PE,-6.8367,-79.9342,82,Hospedaje Calle del Sol
593,luchegorsk,RU,46.4764,134.2569,57,Мираж


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

filtered_hotel_df = hotel_df[hotel_df['Hotel Name']!="No hotel found"]


# Configure the map plot
map_plot_2 = filtered_hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    color="Hotel Name",
    geo = True,
    tiles = "OSM",
    hover_cols=["Humidity","City","Country"],
    frame_width = 800,
    frame_height = 600,
    cmap="Cividis",
    size=30
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)